# Imports

In [27]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import backend
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import SGD

from sklearn import preprocessing as pp
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


## Fix Random Seed For Producibility

In [4]:
seed = 7
np.random.seed(seed)

In [5]:
sonar_df1 = pd.read_csv('sonar.csv', header = None)
sonar_df1

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
5,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R
6,0.0317,0.0956,0.1321,0.1408,0.1674,0.1710,0.0731,0.1401,0.2083,0.3513,...,0.0201,0.0248,0.0131,0.0070,0.0138,0.0092,0.0143,0.0036,0.0103,R
7,0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465,0.2838,...,0.0081,0.0120,0.0045,0.0121,0.0097,0.0085,0.0047,0.0048,0.0053,R
8,0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684,0.1487,...,0.0145,0.0128,0.0145,0.0058,0.0049,0.0065,0.0093,0.0059,0.0022,R
9,0.0164,0.0173,0.0347,0.0070,0.0187,0.0671,0.1056,0.0697,0.0962,0.0251,...,0.0090,0.0223,0.0179,0.0084,0.0068,0.0032,0.0035,0.0056,0.0040,R


In [6]:
enc = pp.LabelEncoder()

sonar_df1.iloc[:,60] = enc.fit_transform(sonar_df1.iloc[:,60])
sonar_df1

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1
5,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,1
6,0.0317,0.0956,0.1321,0.1408,0.1674,0.1710,0.0731,0.1401,0.2083,0.3513,...,0.0201,0.0248,0.0131,0.0070,0.0138,0.0092,0.0143,0.0036,0.0103,1
7,0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465,0.2838,...,0.0081,0.0120,0.0045,0.0121,0.0097,0.0085,0.0047,0.0048,0.0053,1
8,0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684,0.1487,...,0.0145,0.0128,0.0145,0.0058,0.0049,0.0065,0.0093,0.0059,0.0022,1
9,0.0164,0.0173,0.0347,0.0070,0.0187,0.0671,0.1056,0.0697,0.0962,0.0251,...,0.0090,0.0223,0.0179,0.0084,0.0068,0.0032,0.0035,0.0056,0.0040,1


In [15]:
sonar_df = shuffle(sonar_df1)
sonar_df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
31,0.0084,0.0153,0.0291,0.0432,0.0951,0.0752,0.0414,0.0259,0.0692,0.1753,...,0.0236,0.0114,0.0136,0.0117,0.0060,0.0058,0.0031,0.0072,0.0045,1
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,0
60,0.0130,0.0006,0.0088,0.0456,0.0525,0.0778,0.0931,0.0941,0.1711,0.1483,...,0.0078,0.0041,0.0013,0.0011,0.0045,0.0039,0.0022,0.0023,0.0016,1
30,0.0240,0.0218,0.0324,0.0569,0.0330,0.0513,0.0897,0.0713,0.0569,0.0389,...,0.0162,0.0146,0.0093,0.0112,0.0094,0.0054,0.0019,0.0066,0.0023,1
6,0.0317,0.0956,0.1321,0.1408,0.1674,0.1710,0.0731,0.1401,0.2083,0.3513,...,0.0201,0.0248,0.0131,0.0070,0.0138,0.0092,0.0143,0.0036,0.0103,1
128,0.0374,0.0586,0.0628,0.0534,0.0255,0.1422,0.2072,0.2734,0.3070,0.2597,...,0.0118,0.0063,0.0237,0.0032,0.0087,0.0124,0.0113,0.0098,0.0126,0
25,0.0201,0.0026,0.0138,0.0062,0.0133,0.0151,0.0541,0.0210,0.0505,0.1097,...,0.0108,0.0070,0.0063,0.0030,0.0011,0.0007,0.0024,0.0057,0.0044,1
131,0.1150,0.1163,0.0866,0.0358,0.0232,0.1267,0.2417,0.2661,0.4346,0.5378,...,0.0099,0.0065,0.0085,0.0166,0.0110,0.0190,0.0141,0.0068,0.0086,0
42,0.0211,0.0319,0.0415,0.0286,0.0121,0.0438,0.1299,0.1390,0.0695,0.0568,...,0.0090,0.0042,0.0153,0.0106,0.0020,0.0105,0.0049,0.0070,0.0080,1
115,0.0414,0.0436,0.0447,0.0844,0.0419,0.1215,0.2002,0.1516,0.0818,0.1975,...,0.0045,0.0136,0.0113,0.0053,0.0165,0.0141,0.0077,0.0246,0.0198,0


In [18]:
df_train = sonar_df.iloc[:147, :60]
df_train_label = sonar_df.iloc[:147, 60]

df_validation = sonar_df.iloc[147:167, :60]
df_validation_label = sonar_df.iloc[147:167, 60]

df_test = sonar_df.iloc[167:209, :60]
df_test_label = sonar_df.iloc[167:209, 60]

train_data = df_train.to_numpy()
train_labels = df_train_label.to_numpy()

validation_data = df_validation.to_numpy()
validation_labels = df_validation_label.to_numpy()

test_data = df_test.to_numpy()
test_labels = df_test_label.to_numpy()

In [21]:
train_labels

array([1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1])

In [22]:
one_hot_train_labels = to_categorical(train_labels)
one_hot_validation_labels = to_categorical(validation_labels)
one_hot_test = to_categorical(test_labels)

In [23]:
one_hot_train_labels

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.

In [28]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dropout(0.4, input_shape = (60,)),
                                    tf.keras.layers.Dense(64, activation = 'relu', kernel_constraint = max_norm(3)),
                                    tf.keras.layers.Dense(32, activation = 'relu'),
                                    tf.keras.layers.Dense(1, activation = 'sigmoid')])

## Compile Model

In [29]:
sgd = SGD(lr = 0.1, momentum = 0.8, decay = 0.002, nesterov = False)
model.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])

In [31]:
model.fit(train_data , train_labels , epochs = 500, validation_data = (validation_data, validation_labels))

Train on 147 samples, validate on 20 samples
Epoch 1/500
147/147 [==============================] - 4s 28ms/sample - loss: 0.7081 - accuracy: 0.5374 - val_loss: 0.6699 - val_accuracy: 0.6000
Epoch 2/500
147/147 [==============================] - 0s 179us/sample - loss: 0.7073 - accuracy: 0.5374 - val_loss: 0.6926 - val_accuracy: 0.6000
Epoch 3/500
147/147 [==============================] - 0s 223us/sample - loss: 0.6558 - accuracy: 0.5986 - val_loss: 0.6577 - val_accuracy: 0.6000
Epoch 4/500
147/147 [==============================] - 0s 321us/sample - loss: 0.6627 - accuracy: 0.5850 - val_loss: 0.6607 - val_accuracy: 0.6000
Epoch 5/500
147/147 [==============================] - 0s 299us/sample - loss: 0.6592 - accuracy: 0.5850 - val_loss: 0.6679 - val_accuracy: 0.5500
Epoch 6/500
147/147 [==============================] - 0s 245us/sample - loss: 0.6474 - accuracy: 0.6327 - val_loss: 0.6591 - val_accuracy: 0.6000
Epoch 7/500
147/147 [==============================] - 0s 190us/sample - l

Epoch 56/500
147/147 [==============================] - 0s 190us/sample - loss: 0.5720 - accuracy: 0.6871 - val_loss: 0.6411 - val_accuracy: 0.5500
Epoch 57/500
147/147 [==============================] - 0s 163us/sample - loss: 0.5844 - accuracy: 0.6259 - val_loss: 0.6535 - val_accuracy: 0.6000
Epoch 58/500
147/147 [==============================] - 0s 190us/sample - loss: 0.5120 - accuracy: 0.7551 - val_loss: 0.6230 - val_accuracy: 0.5500
Epoch 59/500
147/147 [==============================] - 0s 163us/sample - loss: 0.5226 - accuracy: 0.7279 - val_loss: 0.6267 - val_accuracy: 0.6500
Epoch 60/500
147/147 [==============================] - 0s 190us/sample - loss: 0.5116 - accuracy: 0.7347 - val_loss: 0.6181 - val_accuracy: 0.5500
Epoch 61/500
147/147 [==============================] - 0s 272us/sample - loss: 0.5765 - accuracy: 0.7143 - val_loss: 0.6165 - val_accuracy: 0.5500
Epoch 62/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4998 - accuracy: 0.7755 - val_

147/147 [==============================] - 0s 190us/sample - loss: 0.4533 - accuracy: 0.7823 - val_loss: 0.6015 - val_accuracy: 0.6500
Epoch 112/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4716 - accuracy: 0.7619 - val_loss: 0.6091 - val_accuracy: 0.5500
Epoch 113/500
147/147 [==============================] - 0s 163us/sample - loss: 0.4512 - accuracy: 0.7755 - val_loss: 0.5998 - val_accuracy: 0.7000
Epoch 114/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4970 - accuracy: 0.7551 - val_loss: 0.5986 - val_accuracy: 0.7500
Epoch 115/500
147/147 [==============================] - 0s 218us/sample - loss: 0.4334 - accuracy: 0.7755 - val_loss: 0.6028 - val_accuracy: 0.6500
Epoch 116/500
147/147 [==============================] - 0s 163us/sample - loss: 0.4921 - accuracy: 0.7687 - val_loss: 0.5855 - val_accuracy: 0.7000
Epoch 117/500
147/147 [==============================] - 0s 190us/sample - loss: 0.5020 - accuracy: 0.7755 - val_loss: 0

Epoch 166/500
147/147 [==============================] - 0s 245us/sample - loss: 0.3487 - accuracy: 0.8571 - val_loss: 0.5336 - val_accuracy: 0.7000
Epoch 167/500
147/147 [==============================] - 0s 272us/sample - loss: 0.4658 - accuracy: 0.7891 - val_loss: 0.5446 - val_accuracy: 0.7000
Epoch 168/500
147/147 [==============================] - 0s 272us/sample - loss: 0.4373 - accuracy: 0.7415 - val_loss: 0.5562 - val_accuracy: 0.6500
Epoch 169/500
147/147 [==============================] - 0s 245us/sample - loss: 0.4508 - accuracy: 0.7891 - val_loss: 0.5580 - val_accuracy: 0.6500
Epoch 170/500
147/147 [==============================] - 0s 163us/sample - loss: 0.5088 - accuracy: 0.7415 - val_loss: 0.5783 - val_accuracy: 0.6000
Epoch 171/500
147/147 [==============================] - 0s 163us/sample - loss: 0.4566 - accuracy: 0.7347 - val_loss: 0.5860 - val_accuracy: 0.6000
Epoch 172/500
147/147 [==============================] - 0s 163us/sample - loss: 0.4701 - accuracy: 0.7687

Epoch 221/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4787 - accuracy: 0.7687 - val_loss: 0.5351 - val_accuracy: 0.7500
Epoch 222/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4092 - accuracy: 0.8435 - val_loss: 0.5428 - val_accuracy: 0.7500
Epoch 223/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4716 - accuracy: 0.7619 - val_loss: 0.5571 - val_accuracy: 0.6000
Epoch 224/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4514 - accuracy: 0.7755 - val_loss: 0.5549 - val_accuracy: 0.7500
Epoch 225/500
147/147 [==============================] - 0s 218us/sample - loss: 0.4567 - accuracy: 0.7891 - val_loss: 0.5319 - val_accuracy: 0.8000
Epoch 226/500
147/147 [==============================] - 0s 245us/sample - loss: 0.3745 - accuracy: 0.8299 - val_loss: 0.5975 - val_accuracy: 0.6500
Epoch 227/500
147/147 [==============================] - 0s 245us/sample - loss: 0.4220 - accuracy: 0.8163

Epoch 276/500
147/147 [==============================] - 0s 245us/sample - loss: 0.4304 - accuracy: 0.7755 - val_loss: 0.5486 - val_accuracy: 0.7000
Epoch 277/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4338 - accuracy: 0.7959 - val_loss: 0.5211 - val_accuracy: 0.7000
Epoch 278/500
147/147 [==============================] - 0s 190us/sample - loss: 0.4102 - accuracy: 0.8367 - val_loss: 0.5323 - val_accuracy: 0.7000
Epoch 279/500
147/147 [==============================] - 0s 218us/sample - loss: 0.4662 - accuracy: 0.7755 - val_loss: 0.4952 - val_accuracy: 0.8000
Epoch 280/500
147/147 [==============================] - 0s 218us/sample - loss: 0.3669 - accuracy: 0.8435 - val_loss: 0.5005 - val_accuracy: 0.8500
Epoch 281/500
147/147 [==============================] - 0s 245us/sample - loss: 0.3578 - accuracy: 0.8367 - val_loss: 0.5160 - val_accuracy: 0.7000
Epoch 282/500
147/147 [==============================] - 0s 218us/sample - loss: 0.3978 - accuracy: 0.7823

Epoch 331/500
147/147 [==============================] - 0s 245us/sample - loss: 0.4609 - accuracy: 0.7959 - val_loss: 0.5732 - val_accuracy: 0.6000
Epoch 332/500
147/147 [==============================] - 0s 190us/sample - loss: 0.3590 - accuracy: 0.8503 - val_loss: 0.5841 - val_accuracy: 0.5500
Epoch 333/500
147/147 [==============================] - 0s 299us/sample - loss: 0.3905 - accuracy: 0.8027 - val_loss: 0.5630 - val_accuracy: 0.5500
Epoch 334/500
147/147 [==============================] - 0s 299us/sample - loss: 0.3493 - accuracy: 0.8435 - val_loss: 0.5019 - val_accuracy: 0.7500
Epoch 335/500
147/147 [==============================] - 0s 299us/sample - loss: 0.3545 - accuracy: 0.8571 - val_loss: 0.5092 - val_accuracy: 0.8500
Epoch 336/500
147/147 [==============================] - 0s 245us/sample - loss: 0.3756 - accuracy: 0.8571 - val_loss: 0.5512 - val_accuracy: 0.7500
Epoch 337/500
147/147 [==============================] - 0s 190us/sample - loss: 0.3839 - accuracy: 0.8027

Epoch 386/500
147/147 [==============================] - 0s 218us/sample - loss: 0.4339 - accuracy: 0.8231 - val_loss: 0.5650 - val_accuracy: 0.5500
Epoch 387/500
147/147 [==============================] - 0s 245us/sample - loss: 0.4295 - accuracy: 0.8095 - val_loss: 0.5557 - val_accuracy: 0.6000
Epoch 388/500
147/147 [==============================] - 0s 175us/sample - loss: 0.4214 - accuracy: 0.8435 - val_loss: 0.5315 - val_accuracy: 0.7500
Epoch 389/500
147/147 [==============================] - 0s 332us/sample - loss: 0.3803 - accuracy: 0.8299 - val_loss: 0.5079 - val_accuracy: 0.8500
Epoch 390/500
147/147 [==============================] - 0s 297us/sample - loss: 0.4025 - accuracy: 0.8095 - val_loss: 0.4979 - val_accuracy: 0.8000
Epoch 391/500
147/147 [==============================] - 0s 444us/sample - loss: 0.4245 - accuracy: 0.7755 - val_loss: 0.5387 - val_accuracy: 0.6500
Epoch 392/500
147/147 [==============================] - 0s 301us/sample - loss: 0.4215 - accuracy: 0.7891

Epoch 441/500
147/147 [==============================] - 0s 388us/sample - loss: 0.3345 - accuracy: 0.8095 - val_loss: 0.5188 - val_accuracy: 0.7000
Epoch 442/500
147/147 [==============================] - 0s 299us/sample - loss: 0.3535 - accuracy: 0.8571 - val_loss: 0.5211 - val_accuracy: 0.7500
Epoch 443/500
147/147 [==============================] - 0s 354us/sample - loss: 0.3712 - accuracy: 0.8503 - val_loss: 0.4698 - val_accuracy: 0.8500
Epoch 444/500
147/147 [==============================] - 0s 367us/sample - loss: 0.3897 - accuracy: 0.7755 - val_loss: 0.4826 - val_accuracy: 0.9000
Epoch 445/500
147/147 [==============================] - 0s 467us/sample - loss: 0.3773 - accuracy: 0.7959 - val_loss: 0.5060 - val_accuracy: 0.7000
Epoch 446/500
147/147 [==============================] - 0s 336us/sample - loss: 0.4286 - accuracy: 0.7755 - val_loss: 0.5233 - val_accuracy: 0.7000
Epoch 447/500
147/147 [==============================] - 0s 205us/sample - loss: 0.3923 - accuracy: 0.8095

Epoch 496/500
147/147 [==============================] - 0s 218us/sample - loss: 0.3428 - accuracy: 0.8299 - val_loss: 0.4803 - val_accuracy: 0.8000
Epoch 497/500
147/147 [==============================] - 0s 245us/sample - loss: 0.3155 - accuracy: 0.8639 - val_loss: 0.4799 - val_accuracy: 0.8000
Epoch 498/500
147/147 [==============================] - 0s 253us/sample - loss: 0.3955 - accuracy: 0.8299 - val_loss: 0.4825 - val_accuracy: 0.8000
Epoch 499/500
147/147 [==============================] - 0s 316us/sample - loss: 0.4916 - accuracy: 0.7891 - val_loss: 0.5111 - val_accuracy: 0.7500
Epoch 500/500
147/147 [==============================] - 0s 336us/sample - loss: 0.3964 - accuracy: 0.7891 - val_loss: 0.4961 - val_accuracy: 0.7500
